# Example dataset: BeIR-trec-news-generated-queries
You can download the dataset at [BeIR-trec-news-generated-queries](https://huggingface.co/datasets/BeIR/trec-news-generated-queries)

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
import datasets
from datasets import load_dataset
datasets.logging.set_verbosity_error()

In [3]:
dataset = load_dataset("BeIR/trec-news-generated-queries", split='train')

In [4]:
len(dataset)

1760922

In [10]:
subset = dataset
ids, titles, texts, queries = subset['_id'], subset['title'], subset['text'], subset['query']

## Collect some stats about queries

In [11]:
from collections import defaultdict, Counter

In [12]:
Q = pd.Series(dict(Counter(queries).most_common()))

In [14]:
Q.sort_values(ascending=False).head(20)

who is wsu                         696
what is null                       521
what university is wsu             307
who is the speaker of the house    252
what is wsu                        246
where is wsu                       223
who is the governor of maryland    204
which university is wsu            203
who is ted cruz                    198
who is the mayor of dc             198
who is marco rubio                 174
what is the weather in dc          173
happy hour roundup                 172
weather in dc                      171
what state is wsu                  171
weather in washington dc           168
who is wsu?                        168
who is kellyanne conway            162
weather in dc today                162
who is trump's campaign manager    158
dtype: int64

## Inspect queries

In [15]:
def query_indexes(query):
    return [i for i, x in enumerate(queries) if x == query]
def get_titles(indexes):
    return [titles[i] for i in indexes]
def get_texts(indexes):
    return [texts[i] for i in indexes]

In [24]:
query = 'weather in dc today'
q_i = query_indexes(query)
for text in get_texts(q_i)[:10]:
    print(text[:1000], '\n')

Coming up today in the D.C. region: **Taxicab colors:** There is a public meeting tonight on the uniform color scheme for taxis in the District. **Rare transplant:** A soldier who survived after losing all four limbs in Iraq has received a double-arm transplant at Johns Hopkins Hospital. Doctors at the hospital will hold a news briefing today to provide details of the operation. **Virginia Republicans:** A bill aimed at boosting GOP prospects of winning the White House is up for a committee vote today. ******Tumultuous temperatures:** Temps should climb into the 50s today, and the 60s tomorrow, but the mid-week warm-up won’t last long. **** 

Clouds, rain, fog and gloom undoubtedly have a place in autumn, but to many fans of fall, Friday came closer to the seasonal image. Blue skies — streaked and striped by white clouds, rather than draped in the gray of previous days — helped Friday seem like the week’s best representative of autumn. Not every leaf has turned, but plenty of them blaz

## Subset the data for faster iteration

For the following exercises, we'll use a 10K document subset to speed up computation.

In [ ]:
# Use first 10K documents for faster iteration
subset = dataset.select(range(10000))
doc_ids = subset['_id']
doc_texts = subset['text']
queries = subset['query']

print(f"Working with {len(subset)} documents")

## Exercise 1: Build a Simple TF-IDF Search Index

In this exercise, you'll build a basic retrieval system using scikit-learn's `TfidfVectorizer`.

**Tasks:**
1. Create a TF-IDF vectorizer with English stop words removed
2. Fit and transform the document texts to create a TF-IDF matrix
3. Implement a `search(query, top_k)` function that:
   - Transforms the query using the same vectorizer
   - Computes cosine similarity between the query and all documents
   - Returns the top-K results as a list of `(doc_id, score)` tuples

In [ ]:
# TODO: Build TF-IDF vectorizer and matrix
# vectorizer = TfidfVectorizer(...)
# tfidf_matrix = vectorizer.fit_transform(...)


def search(query, top_k=10):
    """
    Search for documents matching the query.

    Parameters:
    - query: str, the search query
    - top_k: int, number of top results to return

    Returns:
    - list of (doc_id, score) tuples, sorted by score descending
    """
    # TODO: implement search
    pass

In [ ]:
# %load solutions/tfidf_search.py

In [ ]:
# Test the search function
results = search("weather forecast", top_k=5)
for doc_id, score in results:
    idx = doc_ids.index(doc_id)
    print(f"Score: {score:.4f} | {doc_texts[idx][:100]}...")

## Exercise 2: Implement Precision@K

Precision@K measures what fraction of the top-K retrieved documents are relevant.

$$\text{Precision@K} = \frac{|\text{relevant} \cap \text{retrieved@K}|}{K}$$

**Tasks:**
Implement a function that:
- Takes a list of retrieved document IDs (in ranked order)
- Takes a set of relevant document IDs for a query
- Returns the precision at rank K

In [ ]:
def precision_at_k(retrieved_ids, relevant_ids, k):
    """
    Compute Precision@K.

    Parameters:
    - retrieved_ids: list of retrieved document IDs (in ranked order)
    - relevant_ids: set of relevant document IDs for the query
    - k: int, the cutoff rank

    Returns:
    - float, precision@k score
    """
    # TODO: implement precision@k
    pass

In [ ]:
# %load solutions/precision_at_k.py

In [ ]:
# Test precision@k
retrieved = ['doc1', 'doc2', 'doc3', 'doc4', 'doc5']
relevant = {'doc2', 'doc4', 'doc6'}

# Expected: 2/5 = 0.4 (doc2 and doc4 are relevant among top 5)
p5 = precision_at_k(retrieved, relevant, k=5)
print(f"Precision@5: {p5}")
assert p5 == 0.4, f"Expected 0.4, got {p5}"

# Expected: 1/3 = 0.333... (only doc2 is relevant among top 3)
p3 = precision_at_k(retrieved, relevant, k=3)
print(f"Precision@3: {p3:.4f}")
assert abs(p3 - 1/3) < 0.001, f"Expected 0.333, got {p3}"

print("All tests passed!")

## Exercise 3: Implement Mean Reciprocal Rank (MRR)

MRR measures how early the first relevant document appears in the ranked results.

For each query, we compute the reciprocal rank: $\frac{1}{\text{rank of first relevant document}}$

Then MRR is the average across all queries:

$$\text{MRR} = \frac{1}{|Q|} \sum_{i=1}^{|Q|} \frac{1}{\text{rank}_i}$$

**Tasks:**
1. Implement `reciprocal_rank(retrieved_ids, relevant_ids)` for a single query
2. Implement `mean_reciprocal_rank(results)` to average over multiple queries

In [ ]:
def reciprocal_rank(retrieved_ids, relevant_ids):
    """
    Compute the reciprocal rank for a single query.

    Parameters:
    - retrieved_ids: list of retrieved document IDs (in ranked order)
    - relevant_ids: set of relevant document IDs

    Returns:
    - float, reciprocal rank (1/rank of first relevant doc, or 0 if none found)
    """
    # TODO: implement reciprocal rank
    pass


def mean_reciprocal_rank(results):
    """
    Compute Mean Reciprocal Rank over multiple queries.

    Parameters:
    - results: list of (retrieved_ids, relevant_ids) tuples

    Returns:
    - float, MRR score
    """
    # TODO: implement MRR
    pass

In [ ]:
# %load solutions/mrr.py

In [ ]:
# Test reciprocal rank
retrieved = ['doc1', 'doc2', 'doc3', 'doc4', 'doc5']
relevant = {'doc3', 'doc5'}

# First relevant doc is at rank 3 -> RR = 1/3
rr = reciprocal_rank(retrieved, relevant)
print(f"Reciprocal Rank: {rr:.4f}")
assert abs(rr - 1/3) < 0.001, f"Expected 0.333, got {rr}"

# Test with no relevant docs found
rr_none = reciprocal_rank(retrieved, {'doc6', 'doc7'})
print(f"RR (no match): {rr_none}")
assert rr_none == 0.0, f"Expected 0.0, got {rr_none}"

# Test MRR
results = [
    (['a', 'b', 'c'], {'b'}),  # RR = 1/2
    (['a', 'b', 'c'], {'a'}),  # RR = 1/1
    (['a', 'b', 'c'], {'c'}),  # RR = 1/3
]
# MRR = (0.5 + 1.0 + 0.333) / 3 = 0.611
mrr = mean_reciprocal_rank(results)
print(f"MRR: {mrr:.4f}")
assert abs(mrr - (0.5 + 1.0 + 1/3) / 3) < 0.001

print("All tests passed!")

## Exercise 4: Evaluate the Search System

Now let's put everything together and evaluate our TF-IDF search system using the metrics we implemented.

**Tasks:**
1. Sample 100 unique queries from the dataset
2. For each query, find its relevant documents and run search
3. Compute Precision@10 and MRR across all queries
4. Report the average metrics

In [ ]:
# TODO: Sample 100 unique queries
# sample_queries = ...

# TODO: For each query, compute Precision@10 and accumulate MRR results
# k = 10
# precision_scores = []
# mrr_results = []
# for query in sample_queries:
#     ...

# TODO: Report average metrics
# print(f"Average Precision@{k}: ...")
# print(f"Mean Reciprocal Rank: ...")

In [ ]:
# %load solutions/evaluate_search.py